In [78]:
from vapp.package_manager import PackageManager
from vapp.types import ICommand, IContextObject, IFrameData, VApp

In [79]:
%run gptautomator/chat_gpt_automator.py
%run gptautomator/chat_gpt_session.py
from gptautomator.chat_gpt_chrome import ChatGPTChrome


In [82]:
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys


In [80]:
class ChromeVapp(VApp):
    def __init__(self):
        super().__init__()

In [81]:
class WebDriverContextObject(IContextObject):

    
    def _create_value(self):
        chrome_driver = self.webdriver_pkg.webdriver.Chrome()
        return chrome_driver
    
    def __init__(self):
        package_manager = PackageManager()
        self.selenium_pkg = package_manager.get_package('selenium', 'selenium')
        self.webdriver_pkg = package_manager.get_package('selenium', 'selenium.webdriver')
        if not self.selenium_pkg:
            raise Exception("Selenium package is not installed.")
        if not self.webdriver_pkg:
            raise Exception("WebDriver submodule is not available in the Selenium package.")

        self.value = self._create_value()

    def destroy(self):
        if self.value:
            self.value.quit()


In [ ]:
class ClickCommand(ICommand):
    def do_run(self, context_store, *args):
        if len(args) < 1:
            print("XPath argument is missing.")
            return

        xpath = args[0]
        driver = context_store.get("driver").value
        if driver:
            try:
                element = driver.find_element(By.XPATH,xpath)
                element.click()
                print(f"Clicked element with XPath: {xpath}")
            except Exception as e:
                print(f"exception at {e}")
        else:
            print("WebDriver not found in context store.")

In [83]:

class SendKeysCommand(ICommand):
    def do_run(self, context_store, *args):
        if len(args) < 2:
            print("XPath and keys argument are missing.")
            return

        xpath = args[0]
        keys_to_send = args[1]
        driver = context_store.get("driver").value
        if driver:
            try:
                element = driver.find_element(By.XPATH,xpath)
                element.send_keys(keys_to_send)
                print(f"Sent keys '{keys_to_send}' to element with XPath: {xpath}")
            except Exception as e:
                print(f"Exception: {e}")
        else:
            print("WebDriver not found in context store.")


In [84]:
class OpenUrlCommand(ICommand):
    def do_run(self, context_store, *args):
        if len(args) < 1:
            return "XPath argument is missing."

        url = args[0]
        driver = context_store.get("driver").value
        print(driver)
        if driver:
            try:
                driver.get(url)
                return driver
            except Exception as e:
                print(e)
                return e
        else:
            return "WebDriver not found in context store."


In [72]:
class PageTabNavigatedElements(IFrameData):
    def __init__(self):
        self.common = None
        self.io_module = None
        self.time_module = None
        self.csv_package = None
        self._initialize_modules()

    def _initialize_modules(self):
        sel = PackageManager().get_package('selenium','selenium.webdriver.common')
        self.common = sel.webdriver.common
        self.by = self.common.by
        self.keys = self.common.keys
        self.io_module = PackageManager().get_package(None,'io')
        self.time_module = PackageManager().get_package(None,'time')
        self.csv_package = PackageManager().get_package(None,'csv')
        self.javascript =  """
            function getElementXPath(elt) {
                var path = "";
                for (; elt && elt.nodeType == 1; elt = elt.parentNode) {
                    var idx = getElementIdx(elt);
                    var xname = elt.tagName;
                    if (idx > 1) xname += "[" + idx + "]";
                    path = "/" + xname + path;
                }
                return path;
            }

            function getElementIdx(elt) {
                var count = 1;
                for (var sib = elt.previousSibling; sib ; sib = sib.previousSibling) {
                    if (sib.nodeType == 1 && sib.tagName == elt.tagName) count++;
                }
                return count;
            }

            return getElementXPath(arguments[0]);
            """


    def get_frame_data(self, context_store):
        
        if not all([self.common, self.io_module, self.time_module, self.csv_package]):
            print("Failed to import necessary modules.")
            return None

        driver = context_store.get("driver").value
        if not driver:
            print("WebDriver instance not found in context store.")
            return None
        initial_element = driver.find_element(self.by.By.TAG_NAME, "body")
        # List to store data related to focused elements
        data = []
        elements = []
        visited_elements = set()
        headers = ['ELEMENT_INDEX', 'TAG', 'TEXT', 'ID', 'NAME', 'INPUT_TYPE', 'VALUE', 'PLACEHOLDER', 'HREF','XPATH']
        for i in range(50):
            self.time_module.sleep(1)
            initial_element.send_keys(self.keys.Keys.TAB)
            # Get the currently focused element
            focused_element = driver.switch_to.active_element
            elements.insert(i, focused_element)
            # Extract relevant information
            tag_name = focused_element.tag_name
            class_name = focused_element.get_attribute("class")
            input_value = focused_element.get_attribute("value")
            id = focused_element.get_attribute("id")
            placeholder = focused_element.get_attribute("placeholder")
            name = focused_element.get_attribute("name")
            text = focused_element.text if tag_name != "body" else None
            input_type = focused_element.get_attribute("type") if tag_name == "input" else None
            href = focused_element.get_attribute("href") if tag_name == "a" else None
            xpath = "XPATH: "+driver.execute_script(self.javascript, focused_element)
            
            data.append({
                "ELEMENT_INDEX": i,
                "TAG": tag_name,
                "TEXT": text,
                "ID": id,
                "NAME": name,
                "VALUE": input_value,
                "PLACEHOLDER": placeholder,
                "INPUT_TYPE": input_type,
                "HREF": href,
                "XPATH": xpath
            })
            text_small = focused_element.text[:10] if tag_name != "body" else None
            element_identifier = (tag_name, id, text_small, class_name)
            if element_identifier in visited_elements:
                break  # Exit loop if the element has been visited before
            else:
                visited_elements.add(element_identifier)

        csv_buffer = self.io_module.StringIO()
        writer = self.csv_package.DictWriter(csv_buffer, fieldnames=headers)
        writer.writeheader()
        for item in data:
            writer.writerow(item)

        # Get CSV data as a string
        csv_data = csv_buffer.getvalue()

        initial_element.send_keys(self.keys.Keys.CONTROL + 'l')
        
        return csv_data

# Usage:


In [27]:
chrome_vapp = ChromeVapp()
chrome_vapp.add_context_object("driver", WebDriverContextObject())

In [23]:
chrome_vapp.remove_context_object("driver")

In [29]:
chrome_vapp.add_command("open_url",OpenUrlCommand())

In [42]:
chrome_vapp.execute_commands([
    ["open_url","https://www.ibm.com/careers/search"]
])

<selenium.webdriver.chrome.webdriver.WebDriver (session="c833bdbbb9f0944572aaef7407004fb9")>


In [73]:
page_tab_elements = PageTabNavigatedElements()
chrome_vapp.add_frame_data("page_tab_focus",PageTabNavigatedElements())

In [74]:
data = chrome_vapp.get_frame_data("page_tab_focus")
print(data)

ELEMENT_INDEX,TAG,TEXT,ID,NAME,INPUT_TYPE,VALUE,PLACEHOLDER,HREF,XPATH
0,input,,password,session_password,password,,,,/HTML/BODY/DIV/MAIN/DIV[2]/DIV/FORM/DIV[2]/INPUT
1,span,show,password-visibility-toggle,,,,,,/HTML/BODY/DIV/MAIN/DIV[2]/DIV/FORM/DIV[2]/SPAN
2,a,Forgot password?,,,,,,https://www.linkedin.com/checkpoint/rp/request-password-reset?trk=guest_homepage-basic_nav-header-signin,/HTML/BODY/DIV/MAIN/DIV[2]/DIV/FORM/A
3,button,Sign in,,,,,,,/HTML/BODY/DIV/MAIN/DIV[2]/DIV/FORM/DIV[3]/BUTTON
4,a,User Agreement,,,,,,https://www.linkedin.com/legal/user-agreement,/HTML/BODY/DIV/MAIN/DIV[2]/DIV/DIV[2]/P/A
5,a,Privacy Policy,,,,,,https://www.linkedin.com/legal/privacy-policy,/HTML/BODY/DIV/MAIN/DIV[2]/DIV/DIV[2]/P/A[2]
6,a,Cookie Policy,,,,,,https://www.linkedin.com/legal/cookie-policy,/HTML/BODY/DIV/MAIN/DIV[2]/DIV/DIV[2]/P/A[3]
7,iframe,,gsi_888385_902793,,,,,,/HTML/BODY/DIV/MAIN/DIV[2]/DIV/DIV[2]/DIV[2]/DIV/IFRAME
8,iframe,,gsi_888385_902793,,,,,,/HTML/BODY/DIV/MAIN/DIV[2]/DIV/DIV[2]

[41186:40707:0221/134748.215913:ERROR:connection_factory_impl.cc(471)] ConnectionHandler failed with net error: -2


In [69]:
chat_gpt = ChatGPTChrome()

Closing the browser...
You need to manually complete the log-in or the human verification if required.



DevTools listening on ws://127.0.0.1:59751/devtools/browser/30e6e1d5-1450-45ca-8e90-b6c58870c6c5
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


Continuing with the automation process...


In [77]:
chat_gpt.quit()

Closing the browser...


In [70]:
from gptautomator.chat_gpt_session import ChatGPTSession
gptSession = ChatGPTSession(chat_gpt)

In [71]:
import json
prompt = f"""
You will be acting as chrome operating agent.I will give you task,
web page information and my information required to fill any form or may be my login credentials. 
What you will be sending me is commands. Let me explain

I designed a platform where vapps are installed.
Vapp is nothing but a virtual application that holds programmatical access to applications like chrome, file changing.
My Platform has only one vapp which is chrome. I will give you list of commands available in this app.
Each command is list of values as simple as it is. 
First argument will be VAPP ID. For now chrome vapp id is chrome as right now there is only instance of chrome.
Second argument will be command name. Remaining are command related arguments. Ofcourse you will have commands information.
Infact if any command is added, I will be letting you know like below

NEW COMMAND ALERT
    ...Command details...

Installed Vapp and Commands Presently
Vapp : Chrome, id : chrome

Command : Open Url
Examples
["open_url","https://linkedin.com"]
["open_url","https://www.ibm.com"]


When you have to give commands you need to give like below
{
    json.dumps({
        "understood" : True,
        "commands" : [
            ["chrome","open_url","https://example.org"]
        ]
    })
}
YUO ALWAYS GIVE RESPONSE in JSON FROM NOW ON IN CODEBLOCK. 
EVERY JSON RESPONSE SHOULD HAVE MUST FIELD understood: boolean like below if you want to give an acknowlegement
May be lets add your comments, it need not to be exactly like below the comments I think you understood
{
    json.dumps({
        "understood": True,
        "comments": "got it. you could provide with more vapps and its commands. I will help"
    })
}

Obviously if you have understood right. 
"""
gpt_response = gptSession.ask({
    "value" : json.dumps(prompt)
})
gpt_resp_dict = json.loads(gpt_response)
gpt_resp_dict

TypeError: the JSON object must be str, bytes or bytearray, not NoneType

In [58]:
chrome_vapp = ChromeVapp()
chrome_vapp.add_context_object("driver",WebDriverContextObject())

In [75]:
chrome_vapp.remove_context_object("driver")

In [59]:
chrome_vapp.add_command("open_url", OpenUrlCommand())

In [60]:
chrome_vapp.execute_commands([
    ["open_url","https://linkedin.com"]
])

<selenium.webdriver.chrome.webdriver.WebDriver (session="53db58dd9d9f163809059e1c5b4bd85d")>


In [67]:
chrome_vapp.add_command("click_element", ClickCommand())

In [68]:
chrome_vapp.execute_commands([
    ["click_element","//a[contains(text(), 'Sign in')]"]
])

<selenium.webdriver.chrome.webdriver.WebDriver (session="53db58dd9d9f163809059e1c5b4bd85d")>
Clicked element with XPath: //a[contains(text(), 'Sign in')]
